In [78]:
import pandas as pd
import numpy as np

In [79]:
def processXlsx(sheetName):
    print(f'-------开始:{sheetName}-------')
    mData0 = pd.read_excel('../../2023_MCM_Problem_Y_Boats.xlsx', sheet_name=f'{sheetName}')
    mData1 = pd.read_excel('../../2023_MCM_Problem_Y_Boats.xlsx', sheet_name=f'{sheetName}_1')
    mData2 = pd.read_excel('../../2023_MCM_Problem_Y_Boats.xlsx', sheet_name=f'{sheetName}_2')
    mData0.count()

    # 未知值修改为NaN
    for i in ['平均货物吞吐量（吨）', 'GDP（亿美元）', '人均GDP（美元）', '物流总成本占GDP的平均比例']:
        mData2[i] = mData2[i].apply(lambda x: np.NaN if x == '-' else x)

    # 统一厂家型号到一个字段

    def getMakeModel(x):
        return str(x['Make']) + ' ' + str(x['Variant'])

    mData0['Make Variant'] = mData0.apply(lambda x: getMakeModel(x), axis=1)
    mData0.head()

    # 合并原表和帆船参数表
    mData1['型号'] = mData1['型号'].apply(lambda x: x.replace(' ', ''))
    mData0['Make Variant'] = mData0['Make Variant'].apply(lambda x: x.replace(' ', ''))
    mData1.drop_duplicates(keep='first', subset='型号', inplace=True)
    mData = pd.merge(mData0, mData1, how='left', left_on='Make Variant', right_on='型号')
    mData.drop(columns=['Make', 'Variant', '型号'], inplace=True)
    mData.head()

    # 合并地区表
    def x2lower(x):
        try:
            return x.lower().replace(' ', '')
        except:
            return x

    mData2['城市/地区'] = mData2['城市/地区'].apply(lambda x: x2lower(x))
    mData['Country/Region/State '] = mData['Country/Region/State '].apply(lambda x: x2lower(x))
    mData = pd.merge(mData, mData2, how='left', left_on='Country/Region/State ', right_on='城市/地区')
    mData.drop(columns=['城市/地区'], inplace=True)

    print(f'-------结束:{sheetName}-------')
    return mData

In [80]:
def preprocess(mData, tag):
    print(f'-------开始:{tag}-------')
    '''
    数据预处理开始
    -----------1. 去除空值-------------
    '''
    mData.dropna(inplace=True)
    mData.reset_index(inplace=True, drop=True)
    # mData.shape

    # 修正列名
    rawCols = ['Make', 'Variant', 'Length \n(ft)', 'Geographic Region',
               'Country/Region/State ', 'Listing Price (USD)', 'Year', 'Make Variant',
               'LWL (ft)', 'Beam (ft)', 'Draft (ft)', 'Displacement (lbs)',
               'Sail Area (sq ft)', '平均货物吞吐量（吨）', 'GDP（亿美元）', '人均GDP（美元）',
               '物流总成本占GDP的平均比例']

    newCols = ['制造商', '型号', '长度', '地理区域',
               '国家 ', '售价', '年份', '名称',
               '水线长度', '宽度', '吃水', '排水',
               '帆面积', '平均货物吞吐量', 'GDP', '人均GDP',
               '物流总成本占GDP的平均比例']
    # columnsDict = {}
    # for k in range(len(rawCols)):
    #     columnsDict[rawCols[k]] = newCols[k]

    columnsDict = {
        'Make': '制造商',
        'Variant': '型号',
        'Length \n(ft)': '长度',
        'Geographic Region': '地理区域',
        'Country/Region/State ': '国家',
        'Listing Price (USD)': '售价',
        'Year': '年份',
        'Make Variant': '名称',
        'LWL (ft)': '水线长度',
        'Beam (ft)': '宽度',
        'Draft (ft)': '吃水',
        'Displacement (lbs)': '排水',
        'Sail Area (sq ft)': '帆面积',
        '平均货物吞吐量（吨）': '平均货物吞吐量',
        'GDP（亿美元）': 'GDP',
        '人均GDP（美元）': '人均GDP',
        '物流总成本占GDP的平均比例': '物流总成本占GDP的平均比例'
    }
    mData.rename(columns=columnsDict, inplace=True)

    ''''
    新的列名
    '''
    listingPrice = '售价'
    draft = '吃水'
    displacement = '排水'
    sailAria = '帆面积'
    averageCargoThroughput = '平均货物吞吐量'
    gdpPerCapita = '人均GDP'
    year = '年份'

    # mData

    '''
    ---------------2. 3sigma处理异常值-----------------
    '''
    print('原始数据数据', mData.shape)

    # 计算均值和标准差
    mean = mData[listingPrice].mean()
    std = mData[listingPrice].std()

    # 根据 3 Sigma 原则，将数据分为正常值和异常值
    mData = mData[(mData[listingPrice] > mean - 3 * std) & (mData[listingPrice] < mean + 3 * std)]

    # 输出清除异常值后的数据
    print('清除异常值后的数据', mData.shape)

    # xlsxWriter = pd.ExcelWriter(f'./xlsx/合并修正缩放后数据{sheetName}.xlsx')
    # mData.to_excel(xlsxWriter, index=False, sheet_name=f'合并修正{sheetName}')
    # xlsxWriter.save()
    noEncodeMData = mData.copy()

    from sklearn.preprocessing import LabelEncoder

    '''
    ---------------3. 对定性数据进行标签编码-----------------
    '''

    needEncode = [
        '地理区域',
        '国家',
        '名称',
    ]

    # 创建 LabelEncoder 对象
    le = LabelEncoder()

    for lab in needEncode:
        mData[lab] = le.fit_transform(mData[lab])

    '''
    -------------4. 特征缩放---------------
    '''
    mData[year] = mData[year].astype('int64')

    mData[listingPrice] = np.log(mData[listingPrice])
    mData[averageCargoThroughput] = np.log(mData[averageCargoThroughput])
    mData[gdpPerCapita] = np.log(mData[gdpPerCapita])
    mData[displacement] = np.log(mData[displacement])
    mData[sailAria] = mData[sailAria] / 10
    mData[year] = mData[year] - mData[year].min()

    print(f'-------结束:{tag}-------')
    return noEncodeMData, mData

    # mData.to_excel(xlsxWriter, index=False, sheet_name=f'缩放{sheetName}')
    # xlsxWriter.save()


In [81]:
sheetNameList = [
    'Monohulled Sailboats',
    'Catamarans',
]
dataDict = {

}
encodeDataDict = {}
noEncodeDataDict = {}

for sheetName in sheetNameList:
    # 拼接三表之后的数据
    rawData = processXlsx(sheetName)
    # 数据预处理
    noEncodeData, data = preprocess(rawData, f'preprocess[{sheetName}]')
    dataDict[f'未编码{sheetName}'] = noEncodeData.copy()
    dataDict[f'已编码{sheetName}'] = data.copy()
    noEncodeDataDict[sheetName] = noEncodeData.copy()
    encodeDataDict[sheetName] = data.copy()

xlsxWriter = pd.ExcelWriter(f'./xlsx/合并修正缩放后总数据.xlsx')
for k, data in dataDict.items():
    data.to_excel(xlsxWriter, index=False, sheet_name=k)


def combind(dataDict, sheetName, typeCol='类型'):
    combindDf = pd.DataFrame()
    for idx, key in enumerate(dataDict.keys()):
        data = dataDict[key].copy()
        data[typeCol] = idx
        # 合并两个 DataFrame
        combindDf = pd.concat([combindDf, data])

    # 将合并后的数据保存到新的 Excel 文件
    combindDf.to_excel(xlsxWriter, sheet_name=sheetName, index=False)


'''
未编码汇总
已编码汇总
'''
combind(encodeDataDict, '已编码汇总')
combind(noEncodeDataDict, '未编码汇总')

xlsxWriter.save()
print('success')

-------开始:Monohulled Sailboats-------
-------结束:Monohulled Sailboats-------
-------开始:preprocess[Monohulled Sailboats]-------
原始数据数据 (2098, 15)
清除异常值后的数据 (2059, 15)
-------结束:preprocess[Monohulled Sailboats]-------
-------开始:Catamarans-------


D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mData[lab] = le.fit_transform(mData[lab])
D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mData[lab] = le.fit_transform(mData[lab])
D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

-------结束:Catamarans-------
-------开始:preprocess[Catamarans]-------
原始数据数据 (918, 15)
清除异常值后的数据 (911, 15)
-------结束:preprocess[Catamarans]-------


D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mData[lab] = le.fit_transform(mData[lab])
D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mData[lab] = le.fit_transform(mData[lab])
D:\Users\win\AppData\Local\Temp\ipykernel_34788\3963687739.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

success
